In [5]:
import scipy.optimize as opt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import requests


mpl.rc('xtick', direction='in', top=True)
mpl.rc('ytick', direction='in', right=True)
mpl.rc('xtick.minor', visible=True)
mpl.rc('ytick.minor', visible=True)


In [16]:
def Ierr(I):
    Ierr = np.zeros_like(I) # Defining errors based on data table from K6487 manual
    for i in range(len(I)):
        if I[i] > 1e-2:
            Ierr[i] = I[i]*0.1 + 1e-6
        elif (I[i] > 1e-3) & (I[i] < 1e-2):
            Ierr[i] = I[i]*0.1 + 1e-7
        elif (I[i] > 1e-4) & (I[i] < 1e-3):
            Ierr[i] = I[i]*0.1 + 1e-8
        elif (I[i] > 1e-5) & (I[i] < 1e-4):
            Ierr[i] = I[i]*0.1 + 1e-9
        elif (I[i] > 1e-6) & (I[i] < 1e-5):
            Ierr[i] = I[i]*0.1 + 1e-10
        elif (I[i] > 1e-7) & (I[i] < 1e-6):
            Ierr[i] = I[i]*0.1 + 1e-11
        elif (I[i] > 1e-8) & (I[i] < 1e-7):
            Ierr[i] = I[i]*0.15 + 1e-12
        elif (I[i] > 1e-9) & (I[i] < 1e-8):
            Ierr[i] = I[i]*0.15 + 1e-13
        elif (I[i] > 1e-10) & (I[i] < 1e-9):
            Ierr[i] = I[i]*0.2 + 1e-14
        else:
            Ierr[i] = I[i]*0.3 + 1e-15
    return Ierr

def getTxt(folder,file):
  path = "https://raw.githubusercontent.com/nsfuss/Phys472-Band-Gap/refs/heads/main/" + folder + "/" + file + ".txt"
  request = requests.get(path)
  return request.text

In [ ]:
folder = "2025-04-25_Si_20C"
file = "Iavg"

-1.080741166666666707e-08
-1.087371299999999980e-08
-1.107264200000000098e-08
-1.054216500000000068e-08
-8.619212000000000696e-09
-7.094110666666667176e-09
-8.022431000000000708e-09
-9.481219299999999426e-09
-1.034322966666666856e-08
-7.558260199999999149e-09
-6.033157000000001400e-09
-6.563624999999998828e-09
-4.972214166666667171e-09
-6.762548800000000010e-09
-8.751808666666665891e-09
-2.187241999999999797e-09
-3.844957999999999700e-09
9.480000000001363147e-13
1.194509333333332890e-09
-1.979750000000003684e-10
6.726533333333313786e-11
2.520680000000000026e-09
4.708876999999999848e-09
6.167671000000000383e-09
2.852233666666666233e-09
9.350487999999999402e-09
1.346164200000000173e-08
9.416802333333334618e-09
1.213546266666666716e-08
1.160499200000000111e-08
1.538460000000000000e-08
1.870004333333333482e-08
2.208180000000000123e-08
2.360690333333333539e-08
2.665711000000000040e-08
2.698865333333333191e-08
2.957470333333333540e-08
3.700129666666666796e-08
3.361953333333333827e-08
3.93884